In [1]:
"""Functions for downloading and reading MNIST data."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import gzip
import os
import tensorflow.python.platform
import numpy
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
SOURCE_URL = 'http://yann.lecun.com/exdb/mnist/'
def maybe_download(filename, work_directory):
  """Download the data from Yann's website, unless it's already here."""
  if not os.path.exists(work_directory):
    os.mkdir(work_directory)
  filepath = os.path.join(work_directory, filename)
  if not os.path.exists(filepath):
    filepath, _ = urllib.request.urlretrieve(SOURCE_URL + filename, filepath)
    statinfo = os.stat(filepath)
    print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
  return filepath
def _read32(bytestream):
  dt = numpy.dtype(numpy.uint32).newbyteorder('>')
  return numpy.frombuffer(bytestream.read(4), dtype=dt)[0]
# _read32函数：
# dt为 int 32位数据类型 且bit按照大端排序 的数据格式
# 返回值为 从bytestream中读取4个字节，即32位， 且格式为 dt 且取[0]
def extract_images(filename):
  """Extract the images into a 4D uint8 numpy array [index, y, x, depth]."""
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    magic = _read32(bytestream) #第一个字节为magic值
    if magic != 2051:
      raise ValueError(
          'Invalid magic number %d in MNIST image file: %s' %
          (magic, filename))
    num_images = _read32(bytestream) #第二个字节为图片数目 为60000
    rows = _read32(bytestream) #第三个字节为图片的行数  为28
    cols = _read32(bytestream) #第四个字节为图片的列数  为28
    buf = bytestream.read(rows * cols * num_images)  # buf为读出图片的bit流
    data = numpy.frombuffer(buf, dtype=numpy.uint8)  # 将buf流转为int8 格式的数据流
    data = data.reshape(num_images, rows, cols, 1)   # 将data 全部改为(60000*28*28*1) 的size
    return data   #返回
def dense_to_one_hot(labels_dense, num_classes=10):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = numpy.arange(num_labels) * num_classes
  labels_one_hot = numpy.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot
# dense_to_one_hot 函数
# num_labels为labels的个数  index_offset 为扩大num_classes倍数 labels_one_hot 为(60000,10)
# 返回的是(60000,10)的labels
def extract_labels(filename, one_hot=False):  #提取labels的函数 extract_labels
  """Extract the labels into a 1D uint8 numpy array [index]."""
  print('Extracting', filename)
  with gzip.open(filename) as bytestream:
    magic = _read32(bytestream)
    if magic != 2049:
      raise ValueError(
          'Invalid magic number %d in MNIST label file: %s' %
          (magic, filename))
    num_items = _read32(bytestream)   # num_items 为 60000
    buf = bytestream.read(num_items)  # buf为数据
    labels = numpy.frombuffer(buf, dtype=numpy.uint8)
    if one_hot:  #若one_hot为true
      return dense_to_one_hot(labels)   #返回one_hot编码
    return labels  # 返回提取出来的编码
class DataSet(object):
  def __init__(self, images, labels, fake_data=False, one_hot=False,
               dtype=tf.float32):
    """Construct a DataSet.
    one_hot arg is used only if fake_data is true.  `dtype` can be either
    `uint8` to leave the input as `[0, 255]`, or `float32` to rescale into
    `[0, 1]`.
    """
    dtype = tf.as_dtype(dtype).base_dtype
    # dtype 是base_dtype格式
    if dtype not in (tf.uint8, tf.float32):
      raise TypeError('Invalid image dtype %r, expected uint8 or float32' %
                      dtype)
    # 如果dtype 不是int8 或者是 float32
    if fake_data:
      self._num_examples = 10000
      self.one_hot = one_hot
    # 如果fake_data是True，那么 定义两个变量 _num_examples = 10000, one_hot = one_hot的值
    else:
      assert images.shape[0] == labels.shape[0], (
          'images.shape: %s labels.shape: %s' % (images.shape,
                                                 labels.shape))
      self._num_examples = images.shape[0]
    # 否则， 让images.shape[0] = labels.shape[0]
    # 令 _num_examples = images.shape[0]
      # Convert shape from [num examples, rows, columns, depth]
      # to [num examples, rows*columns] (assuming depth == 1)
      assert images.shape[3] == 1
      images = images.reshape(images.shape[0],
                              images.shape[1] * images.shape[2])
    # reshape images   (num*(size*size))
      if dtype == tf.float32:
        # Convert from [0, 255] -> [0.0, 1.0].
        images = images.astype(numpy.float32)
        images = numpy.multiply(images, 1.0 / 255.0)
    # 若dtype 为 float32，那么 images就全部归一化
    self._images = images
    # 定义_images = images  _labels = labels _epochs_completed = 0 _index_in_epoch = 0
    self._labels = labels
    self._epochs_completed = 0
    self._index_in_epoch = 0
  @property
  def images(self):
    return self._images  #property装饰器
  # self._images 的值可以通过Dataset.images 获得。
  @property
  def labels(self):
    return self._labels
  @property
  def num_examples(self):
    return self._num_examples
  @property
  def epochs_completed(self):
    return self._epochs_completed
  # 其余同上
  def next_batch(self, batch_size, fake_data=False):
  # 定义一个函数 next_batch
    """Return the next `batch_size` examples from this data set."""
    if fake_data:
      fake_image = [1] * 784
      if self.one_hot:
        fake_label = [1] + [0] * 9
      else:
        fake_label = 0
      return [fake_image for _ in xrange(batch_size)], [
          fake_label for _ in xrange(batch_size)]
    start = self._index_in_epoch
    self._index_in_epoch += batch_size
  # _index_in_epoch 等于 原来值+batch_size
    if self._index_in_epoch > self._num_examples:
  # 若 _index_in_epoch大于总图片数目
      # Finished epoch
      self._epochs_completed += 1
  # _epochs_completed + 1
      # Shuffle the data
      perm = numpy.arange(self._num_examples)
  # prem 为[0,1,...,59999]
      numpy.random.shuffle(perm)
  # 打乱perm数组的排序
      self._images = self._images[perm]
  # 将图片按照perm数组的编号排序
      self._labels = self._labels[perm]
  # labels同上
      # Start next epoch
      start = 0
      self._index_in_epoch = batch_size
      assert batch_size <= self._num_examples
    end = self._index_in_epoch
  # end = index_in_epoch
    return self._images[start:end], self._labels[start:end]
  # 返回images 和 labels 的[0,batch_size] 部分
def read_data_sets(train_dir, fake_data=False, one_hot=False, dtype=tf.float32):
  class DataSets(object):
    pass            # 定义一个class DataSets
  data_sets = DataSets()   # 实例化这个class， 并命名为data_sets
  if fake_data:     # 若fake_data是true，那么
    def fake():
      return DataSet([], [], fake_data=True, one_hot=one_hot, dtype=dtype)   #定义fake函数
    data_sets.train = fake()    # data_sets的train方法 是fake函数
    data_sets.validation = fake()  # data_sets的validation方法 是fake函数
    data_sets.test = fake()     # data_sets的test方法 是fake函数
    return data_sets            # read_data_sets 函数返回实例 data_sets
  # 若fake_data是false，那么
  TRAIN_IMAGES = 'train-images-idx3-ubyte.gz'
  TRAIN_LABELS = 'train-labels-idx1-ubyte.gz'
  TEST_IMAGES = 't10k-images-idx3-ubyte.gz'
  TEST_LABELS = 't10k-labels-idx1-ubyte.gz'
  VALIDATION_SIZE = 5000
  local_file = maybe_download(TRAIN_IMAGES, train_dir)   # 定义一个local_file   为maybe_download函数的返回值
  # def maybe_download:
  # 返回值为filepath
  train_images = extract_images(local_file)  # train_images 为extract_images 的返回值
  # def extract_images:
  # 返回值为图片bit，且格式为(num*size*size*1)
  local_file = maybe_download(TRAIN_LABELS, train_dir)
  train_labels = extract_labels(local_file, one_hot=one_hot)
  # one_hot默认值为false，返回图片的label，且格式为(num*class)
  local_file = maybe_download(TEST_IMAGES, train_dir)
  # 同理有测试集，测试集的数据num 为10000
  test_images = extract_images(local_file)
  local_file = maybe_download(TEST_LABELS, train_dir)
  test_labels = extract_labels(local_file, one_hot=one_hot)
  validation_images = train_images[:VALIDATION_SIZE]
  # 提出一部分作为验证集，其中验证集的size为5000
  validation_labels = train_labels[:VALIDATION_SIZE]
  train_images = train_images[VALIDATION_SIZE:]
  # 剩余的部分为训练集
  train_labels = train_labels[VALIDATION_SIZE:]
  data_sets.train = DataSet(train_images, train_labels, dtype=dtype)
  # dtype 为32位的浮点数  DataSet实例化 为 data_sets.train
  data_sets.validation = DataSet(validation_images, validation_labels,
                                 dtype=dtype)
  # DataSet 实例化为 data_sets.validation
  data_sets.test = DataSet(test_images, test_labels, dtype=dtype)
  # DataSet 实例化为 data_sets.test
  return data_sets

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
mnist = read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import math
from PIL import Image
from numpy import random
import itertools
from sklearn.metrics import confusion_matrix

In [4]:
compression_nodenum = 784
ph1 = 28
ph2 = 28
obmatrix = 1 * np.random.randn(784, compression_nodenum) + 0
# 将数据全部整理出来变成新的数据集
input1_raw = mnist.train.images
valida1_raw = mnist.validation.images
test1_raw = mnist.test.images
# 原始数据集的数据
input1_compression = np.matmul(input1_raw, obmatrix)
valida1_compression = np.matmul(valida1_raw, obmatrix)
test1_compression = np.matmul(test1_raw, obmatrix)
# 压缩完成后的数据
input1_shape = np.pad(input1_compression, ((0, 0), (0, ph1 * ph2 - compression_nodenum)), 'constant',
                      constant_values=(0))
valida1_shape = np.pad(valida1_compression, ((0, 0), (0, ph1 * ph2 - compression_nodenum)), 'constant',
                       constant_values=(0))
test1_shape = np.pad(test1_compression, ((0, 0), (0, ph1 * ph2 - compression_nodenum)), 'constant', constant_values=(0))
# 重新resize了一下
# 获得了包含压缩信息的新的数据集数据
print(input1_shape.shape)
print(valida1_shape.shape)
print(test1_shape.shape)
# input1_add.shape   (55000, 20, 20)
# valida1_add.shape  (5000, 20, 20)
# test1_add.shape    (10000, 20, 20)

# 将数据集全部打散重新形成新的数据集
perm_input2 = np.arange(valida1_shape.shape[0])
np.random.shuffle(perm_input2)
valida1_image = valida1_shape[perm_input2]
valida1_label = mnist.validation.labels[perm_input2]
# 获得validation数据集的格式
perm_input3 = np.arange(test1_shape.shape[0])
np.random.shuffle(perm_input3)
test1_image = test1_shape[perm_input3]
test1_label = mnist.test.labels[perm_input3]

(55000, 784)
(5000, 784)
(10000, 784)


In [5]:
batch_size = tf.placeholder(tf.int32, [])  # 注意类型必须为 tf.int32
# 在 1.0 版本以后请使用 ：
# keep_prob = tf.placeholder(tf.float32, [])
# batch_size = tf.placeholder(tf.int32, [])
x = tf.placeholder(tf.float32, shape=[None, ph1*ph2], name="x")
# x的大小为[batch_size*784]
y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y_")
# y的大小为[batch_size*10]
x_image = tf.reshape(x, [-1, ph1, ph2, 1])

# 网络参数
filter1_size = 5
filter2_size = 5
W_conv1 = tf.Variable(tf.truncated_normal([filter1_size, filter1_size, 1, 32], stddev=0.1), name='W_conv1')
# W_conv1 产生正太分布随机数，标准差为0.1， shape 为[size*size*depth*num] 卷积核个数为32个
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]), name='b_conv1')
# b_con1 为常数(num,)
L1_conv = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME')
# 第一层做卷积,步长为1
print(L1_conv.shape)
# L1_conv的大小为(batch_size*28*28*num)
L1_relu = tf.nn.relu(L1_conv + b_conv1)
# 第一层做非线形，使用的是RELU
L1_pool = tf.nn.max_pool(L1_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
# ksize 为池化窗口大小 [2,2] strides 为[2,2]
# L1_pool的大小为(batch_size*14*14*num)
print(L1_pool.shape)
# 定义第二个卷积层的variables和ops
W_conv2 = tf.Variable(tf.truncated_normal([filter2_size, filter2_size, 32, 32], stddev=0.1), name='W_conv2')
# W_conv2 shape 为[size*size*depth*num]
b_conv2 = tf.Variable(tf.constant(0.1, shape=[32]), name='b_conv2')
# 生成偏置
L2_conv = tf.nn.conv2d(L1_pool, W_conv2, strides=[1, 1, 1, 1], padding='SAME')
# 做卷积 shape为(batch_size*14*14*num)
L2_relu = tf.nn.relu(L2_conv + b_conv2)
L2_pool = tf.nn.max_pool(L2_relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
# 池化层的shape 为 (batch_size*7*7*num)
# 全连接层
W_fc1 = tf.Variable(tf.truncated_normal([math.ceil(ph1/4)* math.ceil(ph2/4) * 32, 512], stddev=0.1), name='W_fc1')
# 转为（512）
b_fc1 = tf.Variable(tf.constant(0.1, shape=[512]), name='b_fc1')

h_pool2_flat = tf.reshape(L2_pool, [-1, math.ceil(ph1/4)* math.ceil(ph2/4) * 32])
# h_pool2_flat的输出shape为(batch_size,7*7*32)
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
# 输出h_fc1的shape为(batch_size,512)

keep_prob = tf.placeholder(tf.float32, name="keep_prob")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
# dropout层 输出结果为 h_fc1_drop

# readout层
W_fc2 = tf.Variable(tf.truncated_normal([512, 10], stddev=0.1), name='W_fc2')
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]), name='b_fc2')

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

# y_conv为结果   若重复出结果，就需要这个y_conv

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
# 交叉熵设为 softmax_cross_entropy
train_step = tf.train.AdamOptimizer((1e-4)).minimize(cross_entropy)
# 采用Adam的反向传播
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
# 若相等，则加1
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
# 求平均值即为准确率
# 代码持久化

(?, 28, 28, 32)
(?, 14, 14, 32)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
saver = tf.train.Saver()  # 保存模型的saver函数

In [7]:
def count2():
    print (np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()]))

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)
input_count=55000 
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    # 初始化各个variables
    # print ("一共读取了 %s 个测试图像， %s 个标签" % (input_count, input_count))

    # 设置每次训练op的输入个数和迭代次数，这里为了支持任意图片总数，定义了一个余数remainder，譬如，如果每次训练op的输入个数为60，图片总数为150张，则前面两次各输入60张，最后一次输入30张（余数30）
    batch_size = 128
    iterations = 20   # 先用迭代的20次看一下结果
    train_num = 55000

    batches_count = int(train_num / batch_size)  # 可以分为多少批
    remainder = train_num % batch_size
    print("数据训练集分成 %s 批, 前面每批 %s 个数据，最后一批 %s 个数据" % (batches_count + 1, batch_size, remainder))
    # 数据集分成batches_count+1 批


    for i in range(iterations):
        # 在每一次训练之后都需要一个shuffle的过程，大概可以提高训练的结果准确率
        perm_input1 = np.arange(input1_shape.shape[0])
        # prem 为[0,1,...,59999]
        np.random.shuffle(perm_input1)
        # 打乱perm数组的排序
        input1_image = input1_shape[perm_input1]
        input1_label = mnist.train.labels[perm_input1]
            # 获得train数据集的格式
        for n in range(batches_count):
            train_step.run(feed_dict={x: input1_image[n * batch_size:(n + 1) * batch_size],
                                      y_: input1_label[n * batch_size:(n + 1) * batch_size], keep_prob: 0.5})
        # 每次抽出128个进行训练，训练整个数据集
        # 如果remainder > 0，那么就将结果放进去
        # 将placeholder换掉，使用input1和mnist.train.labels去替代x与y_
        # 验证集同上进行训练
        if remainder > 0:
            start_index = batches_count * batch_size;
            train_step.run(
                feed_dict={x: input1_image[start_index:input_count - 1],
                           y_: input1_label[start_index:input_count - 1],
                           keep_prob: 0.5})    # 这个网络有dropout层
# 测试集
        iterate_accuracy = 0
        validation_accuracy = 0
        if i % 5 == 0:
            validation_accuracy = accuracy.eval(feed_dict={x : valida1_image, y_: valida1_label, keep_prob: 1.0})
            # 使用验证集进行验证
            iterate_accuracy = accuracy.eval(feed_dict={x: test1_image, y_: test1_label, keep_prob: 1.0})
                        # 使用验证集进行验证
            print('1-iteration %d: validation_accuracy %s' % (i, validation_accuracy))
            print('2-iteration %d: test_accuracy %s' % (i, iterate_accuracy))
    y_pred = y_conv.eval(feed_dict={x: test1_image, y_: test1_label, keep_prob: 1.0})
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(test1_label, axis=1)
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    saver.save(sess, "./tmp/model.ckpt")    # 利用saver.save 函数保存model的ckpt文件
    print ("save the model")
    print('完成训练!')

count2()

数据训练集分成 430 批, 前面每批 128 个数据，最后一批 88 个数据
1-iteration 0: validation_accuracy 0.7692
2-iteration 0: test_accuracy 0.7665
1-iteration 5: validation_accuracy 0.896
2-iteration 5: test_accuracy 0.8941
1-iteration 10: validation_accuracy 0.9322
2-iteration 10: test_accuracy 0.9304
1-iteration 15: validation_accuracy 0.9498
2-iteration 15: test_accuracy 0.9459
[[ 958    0    4    2    0    5    7    2    2    0]
 [   0 1121    3    2    0    0    3    1    5    0]
 [   9    1  971   14    2    1    5   15   12    2]
 [   1    0    6  966    0   12    2    8   13    2]
 [   0    1    6    0  938    2    7    1    5   22]
 [   4    0    2   20    4  836    4    2   12    8]
 [   7    5    3    0    6   10  920    2    5    0]
 [   0    9   14    4    4    0    0  974    1   22]
 [   4    0    7   16    6   14    5    5  909    8]
 [   8    5    1    9   24    3    1   12   11  935]]
save the model
完成训练!
834922


In [8]:
variable_names = [v.name for v in tf.trainable_variables()]
print(variable_names)

['W_conv1:0', 'b_conv1:0', 'W_conv2:0', 'b_conv2:0', 'W_fc1:0', 'b_fc1:0', 'W_fc2:0', 'b_fc2:0']


In [9]:
tensor_name_list = [tensor.name for tensor in tf.get_default_graph().as_graph_def().node]
for tensor_name in tensor_name_list:
    print(tensor_name,'\n')

Placeholder 

x 

y_ 

Reshape/shape 

Reshape 

truncated_normal/shape 

truncated_normal/mean 

truncated_normal/stddev 

truncated_normal/TruncatedNormal 

truncated_normal/mul 

truncated_normal 

W_conv1 

W_conv1/Assign 

W_conv1/read 

Const 

b_conv1 

b_conv1/Assign 

b_conv1/read 

Conv2D 

add 

Relu 

MaxPool 

truncated_normal_1/shape 

truncated_normal_1/mean 

truncated_normal_1/stddev 

truncated_normal_1/TruncatedNormal 

truncated_normal_1/mul 

truncated_normal_1 

W_conv2 

W_conv2/Assign 

W_conv2/read 

Const_1 

b_conv2 

b_conv2/Assign 

b_conv2/read 

Conv2D_1 

add_1 

Relu_1 

MaxPool_1 

truncated_normal_2/shape 

truncated_normal_2/mean 

truncated_normal_2/stddev 

truncated_normal_2/TruncatedNormal 

truncated_normal_2/mul 

truncated_normal_2 

W_fc1 

W_fc1/Assign 

W_fc1/read 

Const_2 

b_fc1 

b_fc1/Assign 

b_fc1/read 

Reshape_1/shape 

Reshape_1 

MatMul 

add_2 

Relu_2 

keep_prob 

sub/x 

sub 

dropout/Shape 

dropout/random_uniform/min 

drop

In [10]:
sess.close()

In [11]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [12]:
# 从model.ckpt.meta中直接加载已经持久化的图
saver = tf.train.import_meta_graph(
    './tmp/model.ckpt.meta'
)
graph = tf.get_default_graph()

In [13]:
x = graph.get_operation_by_name('x')
y_ = graph.get_operation_by_name('y_')
print(x)
print(y_)

name: "x"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 784
      }
    }
  }
}

name: "y_"
op: "Placeholder"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "shape"
  value {
    shape {
      dim {
        size: -1
      }
      dim {
        size: 10
      }
    }
  }
}



In [14]:
test1_image[0:10].shape

(10, 784)

In [29]:
with tf.Session() as sess:
#    saver.restore(sess, './model/model.ckpt')
    saver.restore(sess, "./tmp/model.ckpt")
    x = sess.graph.get_tensor_by_name('x:0')
    y_ = sess.graph.get_tensor_by_name('y_:0')
    keep_prob =sess.graph.get_tensor_by_name('keep_prob:0')
    output = sess.graph.get_tensor_by_name('add_3:0')
    layer1_output = sess.graph.get_tensor_by_name("Conv2D:0")
    print(layer1_output)
    #print(sess.run(layer1_output, feed_dict={x:test1_image[0:2],keep_prob:1.0}))
    print(sess.run(tf.argmax(output,1), feed_dict={x:valida1_image[0:2],keep_prob:1.0}))
    print(sess.run(tf.argmax(valida1_label[0:2], 1)))

INFO:tensorflow:Restoring parameters from ./tmp/model.ckpt
Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
[1 7]
[1 7]


In [19]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import os
from tensorflow.python import pywrap_tensorflow

checkpoint_path = os.path.join('./tmp/model.ckpt')
reader = pywrap_tensorflow.NewCheckpointReader(checkpoint_path)
var_to_shape_map = reader.get_variable_to_shape_map()

In [30]:
print(var_to_shape_map)

{'beta2_power': [], 'b_fc2': [10], 'b_fc1/Adam': [512], 'b_fc1': [512], 'b_conv2/Adam_1': [32], 'b_fc2/Adam': [10], 'b_conv2/Adam': [32], 'W_conv2/Adam': [5, 5, 32, 32], 'b_conv2': [32], 'W_fc2/Adam': [512, 10], 'W_conv2/Adam_1': [5, 5, 32, 32], 'W_conv2': [5, 5, 32, 32], 'beta1_power': [], 'W_fc2/Adam_1': [512, 10], 'W_conv1': [5, 5, 1, 32], 'W_conv1/Adam': [5, 5, 1, 32], 'b_fc2/Adam_1': [10], 'b_fc1/Adam_1': [512], 'W_fc1/Adam': [1568, 512], 'W_fc1/Adam_1': [1568, 512], 'W_fc1': [1568, 512], 'W_fc2': [512, 10], 'b_conv1': [32], 'b_conv1/Adam': [32], 'W_conv1/Adam_1': [5, 5, 1, 32], 'b_conv1/Adam_1': [32]}


In [31]:
import numpy as np

In [33]:
np.savetxt('layer1_W.txt',W_conv1,fmt='%0.8f')

ValueError: Expected 1D or 2D array, got 4D array instead

In [46]:
W_conv1 = reader.get_tensor('W_conv1')
W_conv1_1 = W_conv1[:,:,:,1]
print(W_conv1_1.shape)
W_conv1_1_1 = W_conv1_1[:,:,0]
print(W_conv1_1_1.shape)

np.savetxt('layer1_W_1.txt',W_conv1_1)
# print(reader.get_tensor('W_conv1'))
#print(W_conv1.shape)

(5, 5, 1)
(5, 5)


ValueError: Expected 1D or 2D array, got 3D array instead